#### Home Credit Default Risk 

#### Imports

In [1]:
# We will import numpy and pandas for data munging
import pandas as pd
import numpy as np

# To encode categorical variables, we will import LabelEncoder
from sklearn.preprocessing import LabelEncoder

# For plotting graphs
import matplotlib
import matplotlib.pyplot as plt # for plotting
%matplotlib inline

# To read, write and operate in a particular directory
import os

# We don't want avoid warnings and make the notebook look neat
import warnings
warnings.filterwarnings('ignore')

#### Listing Data

In [ ]:
# Listing all the filenames that were given by kaggle
print(os.listdir("./data"))

In [ ]:
# Storing data in train and test variables

In [2]:
# Training Data. Inputing into train variable
train = pd.read_csv("./data/application_train.csv") 

# Testing Data. Inputing into test variable 
test = pd.read_csv("./data/application_test.csv")

In [ ]:
# Let's checkout the first 5 rows of our dataset(train and test)

In [ ]:
print(train.shape)
train.head()

In [ ]:
print(test.shape)
test.head()

There is one less column in test data as there is no class(TARGET) variable in the dataset. There are 307511 rows in training data which 307511 loans. 

### Exploratory Data Analysis

In this stage, we will try to use graphs, statistics to understand the data, find missing values, check for any anomalies so on and so forth.

In [ ]:
# We will start with the target variable. Since this is a bi-classification supervised learning problem, 
# we should know how the two classes are divided.
train['TARGET'].value_counts()

This looks like they are not equally divided. Let us examine the ratio.

In [ ]:
ratio = train['TARGET'].value_counts()[1]/train['TARGET'].value_counts()[0]
print("Percentage of observation who made late loan payments = {0:.2f}".format(ratio*100)+" %")
print("Percentage of observation who paid loans on time  = {0:.2f}".format((1-ratio)*100)+" %")

As we can see this is a imbalanced class problem. This problem has to be carefully dealth with. Machine learning algorithms work well when we have a balanced dataset. We don't want or classifier to incorrectly classify people who paid loans on time into people who did not pay their loans on time. In statistics terminology, we would choose the model with fewer False Negatives.

In [ ]:
#plotting a histogram to visualize their difference
plt.figure(figsize = (8,6))
plt.xlabel('Target Variable')
plt.title("Histogram of unique values in target variable")
train['TARGET'].plot.hist()

### Missing Values

Next step is to deal with missing values. We will define a function which will return the percentage of missing values in each variable.

In [ ]:
#  Function that returns percentage of missing values
def missingValues(df):
    
    # returns a series with features and the total number of missing value for each feature
    sum_NAN = df.isnull().sum()  
    
    # returns a series with features and the percentage of number of missing value for each feature
    percent_NAN = sum_NAN/len(df)*100    
    
    # combining both the above into a dataframe
    df_NAN = pd.concat([sum_NAN,percent_NAN], axis = 1) 
    
    # Naming the columns
    df_NAN.columns = ['Sum of Missing Values','Percentage'] 
    
    #Sorting the values in descending order
    df_NAN = df_NAN[df_NAN.iloc[:,1] != 0].sort_values('Percentage' ,ascending = False)
    
    # We will then return a dataframe which gives us a good idea of missing values
    return df_NAN

In [ ]:
mis_val = missingValues(train)
mis_val.shape

In [ ]:
mis_val.head(30)

In [ ]:
mis_val.tail(30)

# Exploring Variables

Taking a look at the different types of variables and the unique values can provide insight into the data.  It can describe the variable whose name is not as descriptive, you can spot outliers and incorrect data, and you can get a feel of the general composition of the data.  Segmenting the different types also makes it easier to clean each type individually and compare the distributions and correlations among different variables.

In [ ]:
# Quick summary statistics
train.describe().T

## Binary Variables

These variables are categorical, but much easier to deal with since there are only two choices.  That means we don't need to do any one hot encoding and and can easily replace the string values with 1 and 0

In [32]:
# Any variable that starts with "Flag"
flag_cols = [col for col in train if col.startswith('FLAG')]

# Any location based variable, except those that are rating based
location_cols = ['REG_REGION_NOT_LIVE_REGION', 'LIVE_REGION_NOT_WORK_REGION', 
                 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 
                 'LIVE_CITY_NOT_WORK_CITY', 'REG_REGION_NOT_WORK_REGION']

# Put everything into one dataframe
bi_df = train[flag_cols + location_cols]

In [37]:
# Make sure that these are actually binary variables, and determine which variables we need to replace
for col in bi_df.columns:
    print(col, bi_df[col].unique())

FLAG_OWN_CAR ['N' 'Y']
FLAG_OWN_REALTY ['Y' 'N']
FLAG_MOBIL [1 0]
FLAG_EMP_PHONE [1 0]
FLAG_WORK_PHONE [0 1]
FLAG_CONT_MOBILE [1 0]
FLAG_PHONE [1 0]
FLAG_EMAIL [0 1]
FLAG_DOCUMENT_2 [0 1]
FLAG_DOCUMENT_3 [1 0]
FLAG_DOCUMENT_4 [0 1]
FLAG_DOCUMENT_5 [0 1]
FLAG_DOCUMENT_6 [0 1]
FLAG_DOCUMENT_7 [0 1]
FLAG_DOCUMENT_8 [0 1]
FLAG_DOCUMENT_9 [0 1]
FLAG_DOCUMENT_10 [0 1]
FLAG_DOCUMENT_11 [0 1]
FLAG_DOCUMENT_12 [0 1]
FLAG_DOCUMENT_13 [0 1]
FLAG_DOCUMENT_14 [0 1]
FLAG_DOCUMENT_15 [0 1]
FLAG_DOCUMENT_16 [0 1]
FLAG_DOCUMENT_17 [0 1]
FLAG_DOCUMENT_18 [0 1]
FLAG_DOCUMENT_19 [0 1]
FLAG_DOCUMENT_20 [0 1]
FLAG_DOCUMENT_21 [0 1]
REG_REGION_NOT_LIVE_REGION [0 1]
LIVE_REGION_NOT_WORK_REGION [0 1]
REG_CITY_NOT_LIVE_CITY [0 1]
REG_CITY_NOT_WORK_CITY [0 1]
LIVE_CITY_NOT_WORK_CITY [0 1]
REG_REGION_NOT_WORK_REGION [0 1]


In [38]:
# Replace the Y/N with 1 and 0
bi_df.FLAG_OWN_CAR = pd.get_dummies(bi_df.FLAG_OWN_CAR)['Y'].astype('int64')
bi_df.FLAG_OWN_REALTY = pd.get_dummies(bi_df.FLAG_OWN_REALTY)['Y'].astype('int64')

## Categorical Variables

These variables are either strings or numbers that do not represent any inherent value and cannot be calculated.

In [40]:
# Any variable begining with 'Name'
name_cols = [col for col in train if col.startswith('NAME')]

# Since these are cyclic, there is no definitive order and are treated as categorical
time_cols = ['WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START']

# All other categorical variables
misc_cat_cols = ['CODE_GENDER', 'OCCUPATION_TYPE', 'ORGANIZATION_TYPE', 
                 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'EMERGENCYSTATE_MODE']

# Put everything into one dataframe
cat_df = train[misc_cat_cols + name_cols + time_cols]

In [41]:
# Make sure that these are actually binary variables, and determine which variables we need to replace
for col in cat_df.columns:
    print(col, cat_df[col].unique())

CODE_GENDER ['M' 'F' 'XNA']
OCCUPATION_TYPE ['Laborers' 'Core staff' 'Accountants' 'Managers' nan 'Drivers'
 'Sales staff' 'Cleaning staff' 'Cooking staff' 'Private service staff'
 'Medicine staff' 'Security staff' 'High skill tech staff'
 'Waiters/barmen staff' 'Low-skill Laborers' 'Realty agents' 'Secretaries'
 'IT staff' 'HR staff']
ORGANIZATION_TYPE ['Business Entity Type 3' 'School' 'Government' 'Religion' 'Other' 'XNA'
 'Electricity' 'Medicine' 'Business Entity Type 2' 'Self-employed'
 'Transport: type 2' 'Construction' 'Housing' 'Kindergarten'
 'Trade: type 7' 'Industry: type 11' 'Military' 'Services'
 'Security Ministries' 'Transport: type 4' 'Industry: type 1' 'Emergency'
 'Security' 'Trade: type 2' 'University' 'Transport: type 3' 'Police'
 'Business Entity Type 1' 'Postal' 'Industry: type 4' 'Agriculture'
 'Restaurant' 'Culture' 'Hotel' 'Industry: type 7' 'Trade: type 3'
 'Industry: type 3' 'Bank' 'Industry: type 9' 'Insurance' 'Trade: type 6'
 'Industry: type 2' 'Transport:

## Ordinal Variables

These are discrete variables whose order matter.  

In [34]:
# There are only two variables that are ordinal
rating_cols = ['REGION_RATING_CLIENT_W_CITY', 'REGION_RATING_CLIENT']

# Put everything into one dataframe
ord_df = train[rating_cols]

In [42]:
# Make sure that these are actually binary variables, and determine which variables we need to replace
for col in ord_df.columns:
    print(col, ord_df[col].unique())

REGION_RATING_CLIENT_W_CITY [2 1 3]
REGION_RATING_CLIENT [2 1 3]


## Numeric Variables

In [44]:
# Anything that begins with 'AMT' represents a monetary value, aside from the AMT_REQ which is a time-based variable
amt_cols = [col for col in train if col.startswith('AMT') and not col.startswith('AMT_REQ')]

# Not sure what the social circle counts are, but assuming they are numerical for now 
cnt_cols = [col for col in train if 'CNT' in col]

# These variables are time-based.  Not sure what they mean, though.
credit_cols = [col for col in train if col.startswith('AMT_REQ_CREDIT_BUREAU')]

# These variables represent number of days since 'X'
days_cols = [col for col in train if col.startswith('DAYS')]

# Additional numeric variables.  Not sure what external source represents
misc_num_cols = ['OWN_CAR_AGE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'REGION_POPULATION_RELATIVE']

# Summary variables, removing some variables that have 'MODE' but are really categorical
summary_cols = [col for col in train if col not in misc_cat_cols and 'MODE' in col or 'MEDI' in col or 'AVG' in col]

# Put everything into one dataframe
num_df = train[amt_cols + cnt_cols + misc_num_cols + days_cols + credit_cols + summary_cols]

In [45]:
# Make sure that these are actually binary variables, and determine which variables we need to replace
for col in num_df.columns:
    print(col, num_df[col].unique())

AMT_INCOME_TOTAL [ 202500.   270000.    67500.  ...,   96768.   546250.5  113062.5]
AMT_CREDIT [  406597.5  1293502.5   135000.  ...,   181989.    743863.5  1391130. ]
AMT_ANNUITY [ 24700.5  35698.5   6750.  ...,  71986.5  58770.   77809.5]
AMT_GOODS_PRICE [  351000.   1129500.    135000.  ...,   453465.    143977.5   743863.5]
CNT_CHILDREN [ 0  1  2  3  4  7  5  6  8  9 11 12 10 19 14]
CNT_FAM_MEMBERS [  1.   2.   3.   4.   5.   6.   9.   7.   8.  10.  13.  nan  14.  12.  20.
  15.  16.  11.]
OBS_30_CNT_SOCIAL_CIRCLE [   2.    1.    0.    4.    8.   10.   nan    7.    3.    6.    5.   12.
    9.   13.   11.   14.   22.   16.   15.   17.   20.   25.   19.   18.
   21.   24.   23.   28.   26.   29.   27.   47.  348.   30.]
DEF_30_CNT_SOCIAL_CIRCLE [  2.   0.   1.  nan   3.   4.   5.   6.   7.  34.   8.]
OBS_60_CNT_SOCIAL_CIRCLE [   2.    1.    0.    4.    8.   10.   nan    7.    3.    6.    5.   12.
    9.   13.   11.   14.   21.   15.   22.   16.   20.   25.   17.   19.
   18.   24.   

AMT_REQ_CREDIT_BUREAU_HOUR [  0.  nan   1.   2.   3.   4.]
AMT_REQ_CREDIT_BUREAU_DAY [  0.  nan   1.   3.   2.   4.   5.   6.   9.   8.]
AMT_REQ_CREDIT_BUREAU_WEEK [  0.  nan   1.   3.   2.   4.   5.   6.   8.   7.]
AMT_REQ_CREDIT_BUREAU_MON [  0.  nan   1.   2.   6.   5.   3.   7.   9.   4.  11.   8.  16.  12.  14.
  10.  13.  17.  24.  19.  15.  23.  18.  27.  22.]
AMT_REQ_CREDIT_BUREAU_QRT [   0.   nan    1.    2.    4.    3.    8.    5.    6.    7.  261.   19.]
AMT_REQ_CREDIT_BUREAU_YEAR [  1.   0.  nan   2.   4.   5.   3.   8.   6.   9.   7.  10.  11.  13.  16.
  12.  25.  23.  15.  14.  22.  17.  19.  18.  21.  20.]
APARTMENTS_AVG [ 0.0247  0.0959     nan ...,  0.0261  0.0731  0.3466]
BASEMENTAREA_AVG [ 0.0369  0.0529     nan ...,  0.274   0.2733  0.636 ]
YEARS_BEGINEXPLUATATION_AVG [ 0.9722  0.9851     nan  0.9811  0.9806  0.9985  0.9881  0.9781  0.9786
  0.9876  0.9861  0.9732  0.9831  0.9697  0.9901  0.9801  0.9762  0.996
  0.998   0.9846  0.9886  0.9354  0.9841  0.9856  0.995

LIVINGAPARTMENTS_AVG [ 0.0202  0.0773     nan ...,  0.2334  0.0111  0.1543]
LIVINGAREA_AVG [ 0.019   0.0549     nan ...,  0.4256  0.6005  0.9279]
NONLIVINGAPARTMENTS_AVG [  0.00000000e+00   3.90000000e-03              nan   1.93000000e-02
   7.70000000e-03   1.90000000e-03   6.56000000e-02   5.80000000e-03
   2.78000000e-01   2.32000000e-02   7.72000000e-02   1.93100000e-01
   3.86000000e-02   1.54000000e-02   2.20100000e-01   1.16000000e-02
   2.70000000e-02   1.62200000e-01   3.09000000e-02   4.25000000e-02
   6.18000000e-02   5.02000000e-02   1.74000000e-02   2.90000000e-02
   2.10000000e-03   1.30000000e-03   6.90000000e-03   1.35000000e-02
   4.10000000e-03   1.50000000e-03   1.27400000e-01   5.79000000e-02
   2.62500000e-01   8.00000000e-04   1.96900000e-01   4.83000000e-02
   4.63000000e-02   4.44000000e-02   1.03000000e-02   5.66000000e-02
   2.83000000e-02   1.15800000e-01   3.47000000e-02   8.11000000e-02
   7.34000000e-02   7.45200000e-01   1.23600000e-01   2.50000000e-03
  

YEARS_BUILD_MEDI [  6.24300000e-01   7.98700000e-01              nan   7.38300000e-01
   9.79900000e-01   8.39000000e-01   7.04800000e-01   7.11500000e-01
   8.32300000e-01   8.12100000e-01   7.71900000e-01   6.78000000e-01
   9.46300000e-01   7.45100000e-01   7.92000000e-01   7.31600000e-01
   8.45700000e-01   0.00000000e+00   7.85300000e-01   8.05400000e-01
   5.63900000e-01   8.52400000e-01   6.64500000e-01   7.65200000e-01
   6.31000000e-01   9.66500000e-01   9.86600000e-01   6.37700000e-01
   9.73200000e-01   8.18900000e-01   6.71300000e-01   7.18200000e-01
   7.58500000e-01   5.70600000e-01   9.26200000e-01   6.91400000e-01
   6.57800000e-01   6.98100000e-01   6.51100000e-01   8.59100000e-01
   5.84000000e-01   7.51800000e-01   9.93300000e-01   8.25600000e-01
   8.72500000e-01   5.97500000e-01   8.85900000e-01   6.10900000e-01
   9.06100000e-01   5.43800000e-01   8.65800000e-01   6.44400000e-01
   8.79200000e-01   9.32900000e-01   7.78600000e-01   4.70000000e-01
   3.29100000e-01